In [1]:
import pandas as pd
from nltk.tokenize import sent_tokenize, word_tokenize
import nltk

# %install_ext https://raw.github.com/cpcloud/ipython-autotime/master/autotime.py
%load_ext autotime

In [2]:
df = pd.read_csv('./shooting_text_snippets.csv', index_col='ia_show_id')

time: 521 ms


In [3]:
df.shape

(34081, 7)

time: 16.7 ms


In [ ]:
# df['tokenized_sents'] = df['snippet'].apply(sent_tokenize)
# df['tokenized_words'] = df['snippet'].apply(word_tokenize)

In [ ]:
# from nltk.corpus import stopwords
# consider removing stop words

In [4]:
def process_content(df, col):
    
    df['tokenized_words'] = df[col].apply(word_tokenize)
    df['tagged'] = df['tokenized_words'].apply(lambda x: nltk.pos_tag(x))
    
    return df

df = process_content(df, 'snippet')

time: 4min 4s


In [5]:
df.to_csv('./df_new.csv')

time: 6.29 s


In [ ]:
# nouns = set()
# for sentence in my_corpus.sents():
#     # each sentence is either a list of words or a list of (word, POS tag) tuples
#     for word, pos in nltk.pos_tag(sentence): # remove the call to nltk.pos_tag if `sentence` is a list of tuples as described above

#         if pos in ['NN', "NNP"]: # feel free to add any other noun tags
#             nouns.add(word)